<h1>Obtaining Manning's n measurements from a lidar point cloud</h1>

This code is relatively straightforward; we are just a using the ```getModelGrid_LAS``` function on each point cloud in our ```input_dir```. 

In [1]:
import os
from tqdm.auto import tqdm
from glob import glob

from PointUtils import getModelGrid_LAS

In [2]:
# PATH TO PYTORCH WEIGHTS
model_dir = r"MRNN_012024.pth"

# PATH WITH LAZ FILES
input_dir = r"data\myPointClouds\\"

# PATH TO OUTPUT TIF FILES
output_dir = r"data\myRoughness\\"

# WILDCARD TO MATCH POINT CLOUD EXTENSIONS
laspy_wildcard = ".laz"

Assemble a list of files to process. Filter by which files already exist in output folder; if any tiles have already been processed, skip them.

In [3]:
full_list = glob(input_dir+"*"+laspy_wildcard)
proc_list = []

for fn in full_list:
    raster_name = f"{output_dir}{os.path.basename(fn)[:-4]}.tif"
    if os.path.exists(raster_name):
        continue
    else:
        proc_list.append(fn)

Process LAZ tiles in directory. By default, the DNN runs on a CUDA-enabled device, however, this can be modified by passing the ```device``` argument to ```getModelGrid_LAS```. Users are encouraged to read through the code and documentation in [PointUtils.py](PointUtils.py).

In [4]:
print(f"Processing {len(proc_list)}")

for fn in tqdm(proc_list, total=len(proc_list)):
    raster_name = f"{output_dir}{os.path.basename(fn)[:-4]}.tif"
    
    model_grid = getModelGrid_LAS(fn, model_dir, ground_class=2,
                                        save=raster_name,
                                        resolution=1,
                                        batch_size=4000)

Processing 1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7822725 [00:00<?, ?it/s]

  0%|          | 0/2400463 [00:00<?, ?it/s]

  0%|          | 0/601 [00:00<?, ?it/s]

At this point, ```output_dir``` will contain a series of roughness tiles, which can be mosaicked together using [GDALVRT](https://gdal.org/drivers/raster/vrt.html) in either the command line or QGIS. The values should thresholded to 0.025 and 0.25, which are the valid measurement regions for the underlying Deep Neural Network. To be input into HEC-RAS, the mosaicked raster must be reclassified to a ```UINT8```, using a tool such as QGIS' [Reclassify by Layer](https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html#reclassify-by-layer) tool. 